In [1]:
import argparse
import os, sys
import time
import copy
import math
import pickle
import statistics

import numpy as np
import pandas as pd
import open3d as o3d
import random
import torchvision
import pytorch3d
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Import pytorch dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.modules.utils import _single, _pair, _triple

# Import toolkits
from summer2022_toolbox.visualization_3D_objects import *
from summer2022_toolbox.preprocessing import *
from summer2022_toolbox.read_object import *
from summer2022_toolbox.model_averaging import *
from summer2022_toolbox.model_PCA import *
from summer2022_toolbox.morphable_model import *
from summer2022_toolbox.model_evaluation import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import os
import torch
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

In [45]:
def prepare_point_cloud(filename, target_vol=1, target_size=2048):
    """
    preprocessing point cloud
    @param filename: mesh file name
    @param target_vol: int
    @param target_size: int
    @return: downsampled np array of shape (3, target_szie)
    """
    # downsample to target size
    pcd = o3d.io.read_point_cloud(filename, format='xyz')
    X = np.asarray(pcd.points)

    idx_lst = random.sample(range(X.shape[0]), target_size)
    X = X[idx_lst]
    
    all_scale = np.array([np.var(X[:, 0]), np.var(X[:, 1]), np.var(X[:, 2])])
    max_scale = np.max(all_scale)
    s = target_vol/max_scale

    centroid = np.mean(X, axis=0)
    X = X - centroid
    X = X * s
    return X.T

In [46]:
#test
filename = "./ShapeNet/car/points/1a3b35be7a0acb2d9f2366ce3e663402.pts"
X = prepare_point_cloud(filename)

print(X.shape)

draw3DPoints(X)

(3, 2048)


In [47]:
def listFileNames(folder):
    """Walk through every files in a directory"""
    filenames = []
    for dirpath, dirs, files in os.walk(folder):
        for filename in files:
            filenames.append(os.path.abspath(os.path.join(dirpath, filename)))

    return filenames

def preprocessAll(filenames):
    """
    filenames: list of filenames (output of listFileNames)
    return list of scaled and downsampled data
    """
    
    pcd_all = []
    for filename in tqdm(filenames):
        pcd = prepare_point_cloud(filename)
        pcd_all.append(pcd)

    return pcd_all

def preprocess_and_save(folder, save_path):
    """
    Unify object sizes and orientation to the reference object
    @param    folder: (in form of (data)/train/... and (data)/test/...)
    @param    ref_name: name of the reference object
    @param    save_path: preprocessed data will be saved here as 'train.txt' and 'test.txt'
    """
    if not os.path.isdir(save_path):
        os.makedirs(save_path)

    folder = folder

    filenames = listFileNames(folder)

    data_all = preprocessAll(filenames)

    print(len(data_all))

    train_idx = random.sample(range(len(data_all)), (int)(len(data_all) * 9 / 10))
    test_idx = [x for x in range(len(data_all)) if x not in train_idx]

    train_all = np.asarray(data_all)[train_idx]
    test_all = np.asarray(data_all)[test_idx]

    train_save_filename = open(save_path + 'train.txt', 'wb')
    pickle.dump(train_all, train_save_filename)

    test_save_filename = open(save_path + 'test.txt', 'wb')
    pickle.dump(test_all, test_save_filename)


In [48]:
preprocess_and_save(folder = './ShapeNet/car/points/',
                    save_path = './processedShapeNet/car_normalized/')

100%|██████████| 1824/1824 [00:06<00:00, 291.29it/s]


1824
